In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4036145/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_0_1,0.975330,-0.264713,0.971333,0.908505,3.471110,187.025744,10.932710,16.349259,0.174316,1.863091,0.894742,1.892433,95.511051,202.056027,"Hidden Size=[12], regularizer=0.2, learning_ra..."
1,model_1_0_2,0.987249,-0.294792,0.982468,0.951475,1.794082,191.473901,6.686072,8.670970,0.095528,1.339433,0.945596,1.360528,96.831013,203.375990,"Hidden Size=[12], regularizer=0.2, learning_ra..."
2,model_1_1_7,0.999659,-0.322290,0.996650,0.998278,0.047979,195.540238,1.277513,0.227045,0.039984,0.219041,0.998545,0.222491,104.073987,210.618963,"Hidden Size=[12], regularizer=0.2, learning_ra..."
3,model_1_1_2,0.998344,-0.339277,0.996054,0.996654,0.232979,198.052351,1.504789,0.441132,0.158325,0.482679,0.992935,0.490281,100.913611,207.458587,"Hidden Size=[12], regularizer=0.2, learning_ra..."
4,model_1_0_4,0.992816,-0.339934,0.985903,0.971943,1.010783,198.149422,5.376205,5.013498,0.056058,1.005377,0.969349,1.021211,97.978549,204.523525,"Hidden Size=[12], regularizer=0.2, learning_ra..."
5,model_1_1_9,0.999622,-0.362375,0.996575,0.998038,0.053134,201.468015,1.306135,0.258688,0.037243,0.230509,0.998389,0.234139,103.869870,210.414846,"Hidden Size=[12], regularizer=0.2, learning_ra..."
6,model_1_2_2,0.999001,-0.515441,0.997005,0.996439,0.140538,224.103406,1.142187,0.619484,0.042697,0.374885,0.995738,0.380789,101.924549,208.469525,"Hidden Size=[12], regularizer=0.2, learning_ra..."
7,model_1_2_6,0.999156,-0.535264,0.996885,0.997010,0.118809,227.034873,1.187771,0.520164,0.040890,0.344687,0.996397,0.350115,102.260476,208.805452,"Hidden Size=[12], regularizer=0.2, learning_ra..."
8,model_1_2_9,0.999368,-0.573853,0.996323,0.997704,0.088870,232.741426,1.402200,0.399457,0.037382,0.298111,0.997305,0.302806,102.841160,209.386136,"Hidden Size=[12], regularizer=0.2, learning_ra..."
